In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.56)
import gc

import os, sys, random, math

import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm
import itertools
from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix
import lightgbm as lgb

In [2]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
%%time

print('Loading data...')

train = pd.read_csv('E-Sun_Credit_Card_Fraud_Data/train.csv', index_col='txkey')
print('\tSuccessfully loaded train!')

test = pd.read_csv('E-Sun_Credit_Card_Fraud_Data/test.csv', index_col='txkey')
print('\tSuccessfully loaded test!')

print('Data was successfully loaded!\n')

print("Train shape : "+str(train.shape))
print("Test shape  : "+str(test.shape))

Loading data...
	Successfully loaded train!
	Successfully loaded test!
Data was successfully loaded!

Train shape : (1521787, 22)
Test shape  : (421665, 21)
Wall time: 4.12 s


In [4]:
def date_time_split(dataframe):
    dataframe['locdt_M'] = dataframe['locdt'].map({1:1,2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1,11:1,12:1,13:1,14:1,15:1,
                                                   16:1,17:1,18:1,19:1,20:1,21:1,22:1,23:1,24:1,25:1,26:1,27:1,28:1,29:1,30:1,31:1,
                                                   32:2,33:2,34:2,35:2,36:2,37:2,38:2,39:2,40:2,41:2,42:2,43:2,44:2,45:2,
                                                   46:2,47:2,48:2,49:2,50:2,51:2,52:2,53:2,54:2,55:2,56:2,57:2,58:2,59:2,60:2,61:2,
                                                   62:3,63:3,64:3,65:3,66:3,67:3,68:3,69:3,70:3,71:3,72:3,73:3,74:3,75:3,
                                                   76:3,77:3,78:3,79:3,80:3,81:3,82:3,83:3,84:3,85:3,86:3,87:3,88:3,89:3,90:3,91:3,92:3,
                                                   93:4,94:4,95:4,96:4,97:4,98:4,99:4,100:4,101:4,102:4,103:4,104:4,105:4,106:4,107:4,
                                                   108:4,109:4,110:4,111:4,112:4,113:4,114:4,115:4,116:4,117:4,118:4,119:4,120:4})
    
    dataframe['locdt_W'] = dataframe['locdt'].map({1:1,2:1,3:1,4:1,5:1,6:1,7:1,
                                                   8:2,9:2,10:2,11:2,12:2,13:2,14:2,
                                                   15:3,16:3,17:3,18:3,19:3,20:3,21:3,
                                                   22:4,23:4,24:4,25:4,26:4,27:4,28:4,
                                                   29:5,30:5,31:5,32:5,33:5,34:5,35:5,
                                                   36:6,37:6,38:6,39:6,40:6,41:6,42:6,
                                                   43:7,44:7,45:7,46:7,47:7,48:7,49:7,
                                                   50:8,51:8,52:8,53:8,54:8,55:8,56:8,
                                                   57:9,58:9,59:9,60:9,61:9,62:9,63:9,
                                                   64:10,65:10,66:10,67:10,68:10,69:10,70:10,
                                                   71:11,72:11,73:11,74:11,75:11,76:11,77:11,
                                                   78:12,79:12,80:12,81:12,82:12,83:12,84:12,
                                                   85:13,86:13,87:13,88:13,89:13,90:13,91:13,
                                                   92:14,93:14,94:14,95:14,96:14,97:14,98:14,
                                                   99:15,100:15,101:15,102:15,103:15,104:15,105:15,
                                                   106:16,107:16,108:16,109:16,110:16,111:16,112:16,
                                                   113:17,114:17,115:17,116:17,117:17,118:17,119:17,
                                                   120:18})
    
    dataframe['loct_hour'] = (dataframe['loctm']//10000).astype(int)
    gc.collect()
    
    return dataframe

train = date_time_split(train)
test = date_time_split(test)

In [5]:
train['txn_info'] = train['ecfg'].astype(str)+train['flg_3dsmk'].astype(str)+train['flbmk'].astype(str)
test['txn_info'] = test['ecfg'].astype(str)+test['flg_3dsmk'].astype(str)+test['flbmk'].astype(str)

cat_columns = ['ecfg','flbmk','flg_3dsmk','insfg','ovrlt','txn_info']

for col in cat_columns:
    train[col] = train[col].fillna('unseen_before_label')
    test[col]  = test[col].fillna('unseen_before_label')
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[col]) + list(test[col]))
    train[col] = lbl.transform(train[col])
    test[col]  = lbl.transform(test[col])

train.fillna(-999,inplace = True)
test.fillna(-999,inplace = True)

In [6]:
# check amount
train['conam_check'] = np.where(train['conam'].isin(test['conam']), 1, 0)
test['conam_check']  = np.where(test['conam'].isin(train['conam']), 1, 0)

# check cano
train['cano_check'] = np.where(train['cano'].isin(test['cano']), 1, 0)
test['cano_check']  = np.where(test['cano'].isin(train['cano']), 1, 0)

In [7]:
train_len = len(train)
temp_train = train.copy()
temp_test = test.copy()

feq_cols = ['mchno','stocn','csmcu','etymd','stscd','txn_info',]
for col in feq_cols:
    train_test_all = pd.concat([temp_train[['cano','conam',col]],temp_test[['cano','conam',col]]]) 
    
    new_col_name = 'cano_'+col+'_fq'
    train_test_all[new_col_name] = train_test_all.groupby(['cano',col])['conam'].transform('count')
    train[new_col_name] = train_test_all[:train_len][new_col_name].tolist()
    test[new_col_name] = train_test_all[train_len:][new_col_name].tolist()
    
    print("'"+new_col_name+"'",', ',end='')

'cano_mchno_fq' , 'cano_stocn_fq' , 'cano_csmcu_fq' , 'cano_etymd_fq' , 'cano_stscd_fq' , 'cano_txn_info_fq' , 

In [8]:
feq_cols = ['locdt','loct_hour']
for col in feq_cols:
    train_test_all = pd.concat([temp_train[['cano','conam',col]],temp_test[['cano','conam',col]]]) 
    
    new_col_name = col+'_cano_count'
    train_test_all[new_col_name] = train_test_all.groupby([col,'cano'])['conam'].transform('count')
    train[new_col_name] = train_test_all[:train_len][new_col_name].tolist()
    test[new_col_name] = train_test_all[train_len:][new_col_name].tolist()
    
    print("'"+new_col_name+"'",', ',end='')

'locdt_cano_count' , 'loct_hour_cano_count' , 

In [9]:
feq_cols = ['mchno','stocn','csmcu','etymd','stscd','txn_info']
for col in feq_cols:
    train_test_all = pd.concat([train[['cano',col]],test[['cano',col]]]) 
    
    new_col_name = col+'_cano'+'_fq'
    train_test_all[new_col_name] = train_test_all.groupby([col])['cano'].transform('count')
    train[new_col_name] = train_test_all[:train_len][new_col_name].tolist()
    test[new_col_name] = train_test_all[train_len:][new_col_name].tolist()
    
    print("'"+new_col_name+"'",', ',end='')

'mchno_cano_fq' , 'stocn_cano_fq' , 'csmcu_cano_fq' , 'etymd_cano_fq' , 'stscd_cano_fq' , 'txn_info_cano_fq' , 

In [10]:
feq_cols = ['mchno','stocn','csmcu']
for col in feq_cols:
    train_test_all = pd.concat([train[['locdt','cano',col]],test[['locdt','cano',col]]]) 
    
    new_col_name = 'day_'+col+'_cano'+'_fq'
    train_test_all[new_col_name] = train_test_all.groupby(['locdt',col])['cano'].transform('count')
    train[new_col_name] = train_test_all[:train_len][new_col_name].tolist()
    test[new_col_name] = train_test_all[train_len:][new_col_name].tolist()
    
    print("'"+new_col_name+"'",', ',end='')

'day_mchno_cano_fq' , 'day_stocn_cano_fq' , 'day_csmcu_cano_fq' , 

In [11]:
train_test_all = pd.concat([temp_train[['locdt_W','acqic', 'mchno', 'scity','csmcu','conam']],temp_test[['locdt_W','acqic', 'mchno', 'scity','csmcu','conam']]])
train_test_all['past_w_same_store'] = train_test_all.groupby(['locdt_W','acqic', 'mchno','scity','csmcu',])['conam'].transform('count')
train['past_w_same_store'] = train_test_all[:train_len].past_w_same_store.tolist()
test['past_w_same_store'] = train_test_all[train_len:].past_w_same_store.tolist()

train['past_w_same_store'] = np.where(train['past_w_same_store']>=10, 10, train['past_w_same_store'])
test['past_w_same_store']  = np.where(test['past_w_same_store']>=10, 10, test['past_w_same_store'])

train['past_w_same_store'] = np.where(((train['past_w_same_store']>=5)&(train['past_w_same_store']<10)), 5, train['past_w_same_store'])
test['past_w_same_store'] = np.where(((test['past_w_same_store']>=5)&(test['past_w_same_store']<10)), 5, test['past_w_same_store'])

train['past_w_same_store'] = np.where(((train['past_w_same_store']>1)&(train['past_w_same_store']<5)), 2, train['past_w_same_store'])
test['past_w_same_store'] = np.where(((test['past_w_same_store']>1)&(test['past_w_same_store']<5)), 2, test['past_w_same_store'])

train['past_w_same_store'] = train['past_w_same_store'].astype(str)
test['past_w_same_store'] = test['past_w_same_store'].astype(str)

In [12]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            dt_df[col] = dt_df[col].astype(float)  

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])

            del dt_df['temp_min'],dt_df['temp_max']
    return dt_df


train_test_all = pd.concat([temp_train[['cano','locdt']],temp_test[['cano','locdt']]],ignore_index=True,sort=False)
train_test_all.reset_index(inplace=True, drop=True)
train_test_all = train_test_all.sort_values('locdt')
train_test_all.drop_duplicates('cano',keep='first',inplace=True)

train_test_all.set_index('cano',inplace=True)
cano_date = train_test_all['locdt'].to_dict()

train['first_cano_dt'] = train['cano'].map(cano_date)
test['first_cano_dt'] = test['cano'].map(cano_date)

train = values_normalization(train, ['locdt_W'], ['first_cano_dt'])
test = values_normalization(test, ['locdt_W'], ['first_cano_dt'])

del temp_train, temp_test, train_test_all

In [13]:
# FREQUENCY ENCODE
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print("'"+nm+"'",', ',end='')
        
# LABEL ENCODE
def encode_LE(col,train=train,test=test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print("'"+nm+"'",', ',end='')

# COMBINE FEATURES
def encode_CB(col1,col2,df1=train,df2=test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print("'"+nm+"'",', ',end='')

# GROUP AGGREGATION MEAN AND STD
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=train, test_df=test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-999,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-999,inplace=True)
                    test_df[new_col_name].fillna(-999,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')

# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, train_df=train, test_df=test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print("'"+col+'_'+main_column+'_ct'+"'",', ',end='')

# COUNT ENCODE            
def encode_CT(uids,train=train,test=test):
    for col in uids:
        train[col + '_count_full'] = train[col].map(pd.concat([train[col], test[col]]).value_counts(dropna=False))
        test[col + '_count_full'] = test[col].map(pd.concat([train[col], test[col]]).value_counts(dropna=False))
        print("'"+col+ '_count_full'+"'",', ',end='')

def encode_TG(uids,train=train,test=test):
    for col in uids:
        temp_dict = train.groupby([col])['fraud_ind'].agg(['mean']).reset_index().rename(columns={'mean': col+'_target_mean'})
        temp_dict.index = temp_dict[col].values
        temp_dict = temp_dict[col+'_target_mean'].to_dict()
    
        train[col+'_target_mean'] = train[col].map(temp_dict)
        test[col+'_target_mean']  = test[col].map(temp_dict)
        print("'"+col+'_target_mean'+"'",', ',end='')

In [14]:
encode_AG(['conam'],['cano', 'bacno', 'mcc', 'mchno', 'acqic',],['mean','std'],usena=True)

encode_FE(train,test,['cano', 'bacno', 'mcc', 'mchno', 'acqic',])

encode_FE(train,test,['etymd', 'stscd', 'txn_info', 'contp', 'hcefg', 'iterm',])

encode_TG(['etymd', 'stscd', 'txn_info'],train,test)

encode_CB('mcc','mchno')
encode_CB('scity','stocn')
encode_CB('mchno','stocn')
#encode_FE(train,test,['mcc_mchno','scity_stocn','mchno_stocn'])
encode_CT(['mcc_mchno','scity_stocn','mchno_stocn'],train,test,)

encode_CB('csmcu','stscd')
encode_CB('acqic','etymd')
encode_FE(train,test,['acqic_etymd','csmcu_stscd'])
#encode_CT(['acqic_etymd','csmcu_stscd'],train,test,)

encode_CB('stscd','etymd')
encode_CB('txn_info','etymd')

'conam_cano_mean' , 'conam_cano_std' , 'conam_bacno_mean' , 'conam_bacno_std' , 'conam_mcc_mean' , 'conam_mcc_std' , 'conam_mchno_mean' , 'conam_mchno_std' , 'conam_acqic_mean' , 'conam_acqic_std' , 'cano_FE' , 'bacno_FE' , 'mcc_FE' , 'mchno_FE' , 'acqic_FE' , 'etymd_FE' , 'stscd_FE' , 'txn_info_FE' , 'contp_FE' , 'hcefg_FE' , 'iterm_FE' , 'etymd_target_mean' , 'stscd_target_mean' , 'txn_info_target_mean' , 'mcc_mchno' , 'scity_stocn' , 'mchno_stocn' , 'mcc_mchno_count_full' , 'scity_stocn_count_full' , 'mchno_stocn_count_full' , 'csmcu_stscd' , 'acqic_etymd' , 'acqic_etymd_FE' , 'csmcu_stscd_FE' , 'stscd_etymd' , 'txn_info_etymd' , 

In [15]:
train['uid'] = train['cano'].astype(str)+'_'+train['bacno'].astype(str)
test['uid'] = test['cano'].astype(str)+'_'+test['bacno'].astype(str)

train['uid2'] = train['uid'].astype(str)+'_'+train['mchno'].astype(str)
test['uid2'] = test['uid'].astype(str)+'_'+test['mchno'].astype(str)

encode_FE(train,test,['uid'])
encode_AG(['conam'],['uid'],['mean','std'],usena=True)

encode_AG(['mcc_mchno','scity_stocn','mchno_stocn'],['uid'],['mean','std'],usena=True)
encode_AG(['etymd','stscd','txn_info'],['uid'],['mean'],fillna=True,usena=True)

encode_AG2(['conam', 'scity_stocn', 'mcc_mchno', 'mchno_stocn'], ['uid'], train_df=train, test_df=test)

encode_FE(train,test,['uid2'])
encode_AG(['conam'],['uid2'],['mean','std'],usena=True)

encode_AG(['acqic_etymd','csmcu_stscd'],['uid2'],['mean','std'],usena=True)
encode_AG(['stscd_etymd','txn_info_etymd'],['uid'],['std'],fillna=True,usena=True)
encode_AG2(['acqic_etymd', 'csmcu_stscd'], ['uid'], train_df=train, test_df=test)

'uid_FE' , 'conam_uid_mean' , 'conam_uid_std' , 'mcc_mchno_uid_mean' , 'mcc_mchno_uid_std' , 'scity_stocn_uid_mean' , 'scity_stocn_uid_std' , 'mchno_stocn_uid_mean' , 'mchno_stocn_uid_std' , 'etymd_uid_mean' , 'stscd_uid_mean' , 'txn_info_uid_mean' , 'uid_conam_ct' , 'uid_scity_stocn_ct' , 'uid_mcc_mchno_ct' , 'uid_mchno_stocn_ct' , 'uid2_FE' , 'conam_uid2_mean' , 'conam_uid2_std' , 'acqic_etymd_uid2_mean' , 'acqic_etymd_uid2_std' , 'csmcu_stscd_uid2_mean' , 'csmcu_stscd_uid2_std' , 'stscd_etymd_uid_std' , 'txn_info_etymd_uid_std' , 'uid_acqic_etymd_ct' , 'uid_csmcu_stscd_ct' , 

In [16]:
train['conam'] = np.log(train['conam'])
test['conam'] = np.log(test['conam'])

In [17]:
%%time
from sklearn.decomposition import FastICA, SparsePCA, TruncatedSVD

cat_cols = ['cano', 'bacno', 'mcc', 'mchno', 'acqic', 'stocn', 'scity', 'csmcu', 
            'etymd', 'stscd', 'txn_info', 'contp', 'hcefg', 'iterm', 'ovrlt']
n_comp = 5

temp_train = train[cat_cols].copy()
temp_test = test[cat_cols].copy()

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica_results_train = ica.fit_transform(temp_train)
ica_results_test = ica.transform(temp_test)

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_results_train = tsvd.fit_transform(temp_train)
tsvd_results_test = tsvd.transform(temp_test)

# SparsePCA
spca = SparsePCA(n_components=n_comp, random_state=42)
spca_results_train = spca.fit_transform(temp_train)
spca_results_test = spca.transform(temp_test)

for i in range(1, n_comp + 1):

    train['tSVD_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tSVD_' + str(i)] = tsvd_results_test[:, i - 1]

    train['ica_' + str(i)] = ica_results_train[:, i - 1]
    test['ica_' + str(i)] = ica_results_test[:, i - 1]
    
    train['spca_' + str(i)] = ica_results_train[:, i - 1]
    test['spca_' + str(i)] = ica_results_test[:, i - 1]

del temp_train, temp_test, ica_results_train, ica_results_test, tsvd_results_train, tsvd_results_test, spca_results_train, spca_results_test
gc.collect()

C:\Users\Arctic Yen\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\decomposition\sparse_pca.py:170: DeprecationWarning: normalize_components=False is a backward-compatible setting that implements a non-standard definition of sparse PCA. This compatibility mode will be removed in 0.22.
  DeprecationWarning)


Wall time: 19.5 s


0

In [18]:
class BetaEncoder(object):
        
    def __init__(self, group):
        
        self.group = group
        self.stats = None
        
    # get counts from df
    def fit(self, df, target_col):
        self.prior_mean = np.mean(df[target_col])
        stats = df[[target_col, self.group]].groupby(self.group)
        stats = stats.agg(['sum', 'count'])[target_col]    
        stats.rename(columns={'sum': 'n', 'count': 'N'}, inplace=True)
        stats.reset_index(level=0, inplace=True)           
        self.stats = stats
        
    # extract posterior statistics
    def transform(self, df, stat_type, N_min=1):
        
        df_stats = pd.merge(df[[self.group]], self.stats, how='left')
        n = df_stats['n'].copy()
        N = df_stats['N'].copy()
        
        # fill in missing
        nan_indexs = np.isnan(n)
        n[nan_indexs] = self.prior_mean
        N[nan_indexs] = 1.0
        
        # prior parameters
        N_prior = np.maximum(N_min-N, 0)
        alpha_prior = self.prior_mean*N_prior
        beta_prior = (1-self.prior_mean)*N_prior
        
        # posterior parameters
        alpha = alpha_prior + n
        beta =  beta_prior + N-n
        
        # calculate statistics
        if stat_type=='mean':
            num = alpha
            dem = alpha+beta
                    
        elif stat_type=='mode':
            num = alpha-1
            dem = alpha+beta-2
            
        elif stat_type=='median':
            num = alpha-1/3
            dem = alpha+beta-2/3
        
        elif stat_type=='var':
            num = alpha*beta
            dem = (alpha+beta)**2*(alpha+beta+1)
                    
        elif stat_type=='skewness':
            num = 2*(beta-alpha)*np.sqrt(alpha+beta+1)
            dem = (alpha+beta+2)*np.sqrt(alpha*beta)

        elif stat_type=='kurtosis':
            num = 6*(alpha-beta)**2*(alpha+beta+1) - alpha*beta*(alpha+beta+2)
            dem = alpha*beta*(alpha+beta+2)*(alpha+beta+3)
            
        # replace missing
        value = num/dem
        value[np.isnan(value)] = np.nanmedian(value)
        return value

In [19]:
cat_columns = ['mchno','scity','acqic']

#N_min = 1000  
# encode variables
for col in cat_columns:
    all_data = pd.concat([train[col],test[col]])
    N_min = (len(all_data.unique()))* 0.1
    # fit encoder
    be = BetaEncoder(col)
    be.fit(train, 'fraud_ind')

    # mean
    train[col+'_beta'] = be.transform(train, 'mean', N_min)
    test[col+'_beta']  = be.transform(test,  'mean', N_min)
    
    del all_data, N_min

In [20]:
for df in [train, test]:
    df = reduce_mem_usage(df)

Mem. usage decreased to 498.61 Mb (57.6% reduction)
Mem. usage decreased to 146.67 Mb (55.7% reduction)


In [21]:
remove_features = ['loctm', 'locdt', 'uid', 'uid2', 'insfg',
                   'first_cano_dt', 'first_cano_dt_locdt_W_min_max', 
                   'locdt_W', 'locdt_M', #'loct_hour', 
                  ]

features_columns = [col for col in list(train) if col not in remove_features]
features_columns.remove('fraud_ind')
target = 'fraud_ind'

In [22]:
for col in features_columns:
    if train[col].dtype=='O':
        print(col)
        train[col] = train[col].fillna('unseen_before_label')
        test[col]  = test[col].fillna('unseen_before_label')
        
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        
        le = preprocessing.LabelEncoder()
        le.fit(list(train[col])+list(test[col]))
        train[col] = le.transform(train[col])
        test[col]  = le.transform(test[col])

past_w_same_store


In [23]:
print('NOW USING THE FOLLOWING',len(features_columns),'FEATURES.')
np.array(features_columns)

NOW USING THE FOLLOWING 121 FEATURES.


array(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg',
       'etymd', 'flbmk', 'flg_3dsmk', 'hcefg', 'iterm', 'mcc', 'mchno',
       'ovrlt', 'scity', 'stocn', 'stscd', 'loct_hour', 'txn_info',
       'conam_check', 'cano_check', 'cano_mchno_fq', 'cano_stocn_fq',
       'cano_csmcu_fq', 'cano_etymd_fq', 'cano_stscd_fq',
       'cano_txn_info_fq', 'locdt_cano_count', 'loct_hour_cano_count',
       'mchno_cano_fq', 'stocn_cano_fq', 'csmcu_cano_fq', 'etymd_cano_fq',
       'stscd_cano_fq', 'txn_info_cano_fq', 'day_mchno_cano_fq',
       'day_stocn_cano_fq', 'day_csmcu_cano_fq', 'past_w_same_store',
       'conam_cano_mean', 'conam_cano_std', 'conam_bacno_mean',
       'conam_bacno_std', 'conam_mcc_mean', 'conam_mcc_std',
       'conam_mchno_mean', 'conam_mchno_std', 'conam_acqic_mean',
       'conam_acqic_std', 'cano_FE', 'bacno_FE', 'mcc_FE', 'mchno_FE',
       'acqic_FE', 'etymd_FE', 'stscd_FE', 'txn_info_FE', 'contp_FE',
       'hcefg_FE', 'iterm_FE', 'etymd_target_mean',

In [24]:
backup_test = test.copy()

In [25]:
test = train[train['locdt_M']==3]
train = train[train['locdt_M']==1]

In [26]:
SEED = 42
seed_everything(SEED)

In [27]:
y = train['fraud_ind']
X = train[features_columns]

X_test_target = pd.DataFrame()
X_test_target['fraud_ind'] = test['fraud_ind']
X_test = test[features_columns]

print ("Size of y_train data : {}" .format(y.shape))
print ("Size of X_train data : {}" .format(X.shape))
print ("Size of X_test data : {}" .format(X_test.shape))

#del train, test
#gc.collect()

Size of y_train data : (533805,)
Size of X_train data : (533805, 121)
Size of test data : (490546, 121)


In [28]:
params = {'objective': 'binary',
          'boosting_type': 'gbdt',
          'metric': 'auc',
          'learning_rate': 0.005,
          'max_depth': -1,
          'tree_learner':'serial',
          'colsample_bytree': 0.7,
          'subsample':0.7,
          'n_estimators':3000,
          'verbosity': -1,
          'seed': SEED,
          #'early_stopping_rounds':100,
         }

In [ ]:
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
auc_score = 0
columns = list(X.columns)
bad_feature = []

for col in columns:

    print(f"column: {col}")
    
    clf = lgb.LGBMClassifier(**params)
    X_train = np.array(X[col]).reshape((-1,1))
    X_vaild = np.array(y).reshape((-1,1))
    X_pred = np.array(X_test[col]).reshape((-1,1))
    clf.fit(X_train, X_vaild)
    
    
    y_preds = clf.predict(X_pred)
    
    X_test_target['pred'] = y_preds
    
    auc_score = roc_auc_score(X_test_target['fraud_ind'].values, X_test_target['pred'])
    print(f"Predict AUC: {auc_score}")
    
    if auc_score < 0.5:
        bad_feature.append(col)
    
    del X_train, X_vaild, X_pred
    gc.collect()

column: acqic
Predict AUC: 0.5
column: bacno
Predict AUC: 0.5
column: cano
Predict AUC: 0.5
column: conam
Predict AUC: 0.5
column: contp
Predict AUC: 0.5
column: csmcu
Predict AUC: 0.5
column: ecfg
Predict AUC: 0.5
column: etymd
Predict AUC: 0.5
column: flbmk
Predict AUC: 0.5
column: flg_3dsmk
Predict AUC: 0.5
column: hcefg
Predict AUC: 0.5
column: iterm
Predict AUC: 0.5
column: mcc
Predict AUC: 0.5
column: mchno
Predict AUC: 0.5
column: ovrlt
Predict AUC: 0.5
column: scity
Predict AUC: 0.5319366448884457
column: stocn
Predict AUC: 0.5
column: stscd
Predict AUC: 0.5
column: loct_hour
Predict AUC: 0.5
column: txn_info
Predict AUC: 0.5
column: conam_check
Predict AUC: 0.5
column: cano_check
Predict AUC: 0.5
column: cano_mchno_fq
Predict AUC: 0.5
column: cano_stocn_fq
Predict AUC: 0.5
column: cano_csmcu_fq
Predict AUC: 0.5
column: cano_etymd_fq


In [ ]:
print(bad_feature)